In [1]:
import os

In [2]:
%pwd

'c:\\Users\\moham\\OneDrive\\Desktop\\deep-learning-project\\Chest-Cancer-Classification-MLOPS-Production-Grade\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\moham\\OneDrive\\Desktop\\deep-learning-project\\Chest-Cancer-Classification-MLOPS-Production-Grade'

In [5]:
import tensorflow as tf

In [6]:
import dagshub
dagshub.init(repo_owner='mohammedasifameenbaig684', repo_name='Chest-Cancer-Classification-MLOPS-Production-Grade', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as mohammedasifameenbaig684

Initialized MLflow to track repo "mohammedasifameenbaig684/Chest-Cancer-Classification-MLOPS-Production-Grade"

Repository mohammedasifameenbaig684/Chest-Cancer-Classification-MLOPS-Production-Grade initialized!

2025/09/21 15:23:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run sedate-cow-44 at: https://dagshub.com/mohammedasifameenbaig684/Chest-Cancer-Classification-MLOPS-Production-Grade.mlflow/#/experiments/0/runs/c51fa68986d945b19de246896fa78746.
2025/09/21 15:23:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/mohammedasifameenbaig684/Chest-Cancer-Classification-MLOPS-Production-Grade.mlflow/#/experiments/0.


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/mohammedasifameenbaig684/Chest-Cancer-Classification-MLOPS-Production-Grade.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-09-21 15:23:13,075: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-21 15:23:13,108: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-21 15:23:13,129: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 65s 7s/step - loss: 14.5276 - accuracy: 0.4314
[2025-09-21 15:24:22,093: INFO: common: json file saved at: scores.json]


2025/09/21 15:24:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-09-21 15:24:25,083: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\moham\AppData\Local\Temp\tmpr2r52gz5\model\data\model\assets
[2025-09-21 15:24:25,855: INFO: builder_impl: Assets written to: C:\Users\moham\AppData\Local\Temp\tmpr2r52gz5\model\data\model\assets]


2025/09/21 15:24:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/09/21 15:25:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
2025/09/21 15:25:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run capable-squid-985 at: https://dagshub.com/mohammedasifameenbaig684/Chest-Cancer-Classification-MLOPS-Production-Grade.mlflow/#/experiments/0/runs/5346e0a7cbee4ad1a45b75a174bca7f1.
2025/09/21 15:25:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/mohammedasifameenbaig684/Chest-Cancer-Classification-MLOPS-Production-Grade.mlflow/#/experiments/0.
